In [1]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [3]:
# load dataset
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

In [10]:
# encode class values as integers
encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [63]:
from keras import models
from keras.layers import Dense
# define baseline model
def baseline_model():
    model = models.Sequential()
    model.add(Dense(4,activation='selu',input_shape=(4,)))
    model.add(Dense(8,activation='relu'))
    model.add(Dense(3,activation='softmax'))
    model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=["accuracy"])
    return model


In [64]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

In [65]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [66]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 97.33% (3.27%)


In [67]:
# Excellent

In [68]:
# define baseline model
def smaller_model():
    model = models.Sequential()
    model.add(Dense(4,activation='selu',input_shape=(4,)))
    model.add(Dense(6,activation='relu'))
    model.add(Dense(3,activation='softmax'))
    model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=["accuracy"])
    return model

In [69]:
estimator = KerasClassifier(build_fn=smaller_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 88.00% (23.63%)


In [70]:
# define baseline model
def larger_model():
    model = models.Sequential()
    model.add(Dense(4,activation='selu',input_shape=(4,)))
    model.add(Dense(16,activation='relu'))
    model.add(Dense(8,activation='relu'))
    model.add(Dense(3,activation='softmax'))
    model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=["accuracy"])
    return model

In [71]:
estimator = KerasClassifier(build_fn=larger_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 96.67% (6.15%)


In [72]:
# overfitting my model
def create_larger():
    model=models.Sequential()
    model.add(Dense(4,activation='selu',input_shape=(4,)))
    model.add(Dense(18,activation='relu'))
    model.add(Dense(34,activation='relu'))
    model.add(Dense(50,activation='relu'))
    model.add(Dense(35,activation='relu'))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(8,activation='relu'))
    model.add(Dense(3,activation='softmax'))
    # looks like i overfitted tooooo much
    model.compile(optimizer='Adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model

In [73]:
estimator = KerasClassifier(build_fn=create_larger, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 97.33% (3.27%)


and tunned 

# Functional API

In [78]:
import keras
def functional_api_model():
    inputs = keras.Input(shape=(4,))
    x=Dense(4,activation='selu')(inputs)
    x=Dense(8,activation='relu')(x)
    outputs =Dense(3,activation='softmax')(x)
    model=keras.Model(inputs,outputs)
    model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [79]:
estimator = KerasClassifier(build_fn=functional_api_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 97.33% (4.42%)


# MODEL SUBCLASSING

In [81]:
import keras
from keras import layers
class MyModel(keras.Model):
    
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 =layers.Dense(4,activation='selu')
        self.dense2 =layers.Dense(8,activation='relu')
        self.dense3 =layers.Dense(3,activation='softmax')
        
    def call(self,inputs):
        x=self.dense1(inputs)
        x=self.dense2(x)
        return self.dense3(x)
    
def mymodel():    
    model = MyModel()
    model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [82]:
estimator = KerasClassifier(build_fn=mymodel, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 98.00% (3.06%)
